# Visualizing statistical estimates and fits

This tutorial shows how to visualize interaction effects included in regression
models such as:

\begin{equation}
  y = \alpha + \beta * x + \gamma * z + \delta * x * z + \epsilon
\end{equation}

Mainly, we use interaction terms to take into account that the effect of the  
independent variable $x$ on the outcome $y$ is contingent on the value of a 
third variable $z$.

Examples:

+   the economic value of patents is contingent on the intellectual property
    regime of a country/set of countries
+   the economic returns of schooling are contingent on the institutional and
    cultural environment of a country
+   the social influence of a Soundcloud user's suggestions/reposts is contingent 
    on followership
+   the influence of job satisfaction on intent to quit is contingent on the 
    size of the employer 

Key reference:

![](images/baron_kenney.png)

# Setup

## Libraries

In [51]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
import statsmodels.api as sm
import statsmodels.formula as smf
import pandas as pd

## Viz options

In [52]:
rc('font',**{'family':'serif','serif':['Avant Garde']})
rc('text', usetex=True)

# Application

## Goal

The working hypothesis is that lower levels of job satisfaction ($x$) increases the
chances of turnover, i.e., intent to quit ($y$).

On top of this, we think the negative relationship between  $x$ and $y$ depends
on the size of the employer. The intuition is that large employers have an 
internal labor market that allows individuals to change job positions (get a
better fit with the task) without quitting the employment relation.

The **goal** is showing how the relationship between job satisfaction and 
intent to quit (as estimated via OLS) changes as firm size increases.

## Data simulation

We simulate multiple datasets involving the following variables:

+   job satisfaction - the extent to which an employee is happy with his/her job
    (let's assume we have Likert scale data; legend: 1 = not at all, 5 = to a 
    great extent)
+   intent to quit - the extent to which an employee considers the possibility 
    to quit the current job (let's assume we have Likert scale data; legend: 1 
    = not at all, 5 = to a great extent)
+   age (in years)
+   organizational tenure (years spent working for the employer)
   
Let's assume that all variables have been transformed into z-scores.
   
Each dataset we simulate reflect the following cohorts of employers:

+   micro firms: 1 - 5 employees
+   small firms: 6 - 25 employees
+   medium firms: 26 - 100 employees
+   large firms: 100 - 500 employees
+   very large firms: 500 + 

In [53]:
# sample size
num_samples = 1000

# variables' mean 
mu = np.repeat(0, 4)

# names
names = ['job_sat', 'int_qui', 'age', 'org_tnr']

### Micro firms

In [54]:
# the desired covariance matrix.
r = np.array([
        [  1.00, -0.40, -0.03,  0.11],
        [ -0.40,  1.00, -0.05, -0.09],
        [ -0.03, -0.05,  1.00,  0.05],
        [  0.11, -0.09,  0.05,  1.00]
    ])

# generate the random samples.
df_1_5 = pd.DataFrame(np.random.multivariate_normal(mu, r, size=num_samples),
                      columns=names)

# expand
df_1_5.loc[:, 'cohort'] = 'micro'
df_1_5.loc[:, 'firm_size'] = np.random.randint(low=1, high=5, size=num_samples)

In [55]:
df_1_5.head()

,job_sat,int_qui,age,org_tnr,cohort,firm_size
0,-1.992759,-0.531744,0.208208,0.844195,micro,2
1,-2.746474,2.511886,-0.970228,0.053533,micro,4
2,-0.957536,-0.381626,0.631114,-0.382642,micro,4
3,0.725831,0.290553,1.543508,-0.499497,micro,1
4,-0.301695,0.276778,1.604164,-0.327978,micro,3


### Small firms

In [56]:
# the desired covariance matrix.
r = np.array([
        [  1.00, -0.30, -0.03,  0.11],
        [ -0.30,  1.00, -0.05, -0.09],
        [ -0.03, -0.05,  1.00,  0.05],
        [  0.11, -0.09,  0.05,  1.00]
    ])

# generate the random samples.
df_6_25 = pd.DataFrame(np.random.multivariate_normal(mu, r, size=num_samples),
                       columns=names)

# expand
df_6_25.loc[:, 'cohort'] = 'large'
df_6_25.loc[:, 'firm_size'] = np.random.randint(low=6,
                                                high=25,
                                                size=num_samples)

In [57]:
df_6_25.head()

,job_sat,int_qui,age,org_tnr,cohort,firm_size
0,-0.107080,1.799623,-0.468387,-2.191865,large,6
1,0.957758,0.035675,1.357635,-1.210960,large,15
2,-0.305284,0.401963,2.074361,-0.872235,large,17
3,-0.452074,0.659556,1.314830,-0.889965,large,11
4,0.270352,-0.606837,-0.908224,0.590164,large,21


### Medium firms

In [58]:
# the desired covariance matrix.
r = np.array([
        [  1.00, -0.25, -0.03,  0.11],
        [ -0.25,  1.00, -0.05, -0.09],
        [ -0.03, -0.05,  1.00,  0.05],
        [  0.11, -0.09,  0.05,  1.00]
    ])

# generate the random samples.
df_26_100 = pd.DataFrame(np.random.multivariate_normal(mu, r, size=num_samples),
                         columns=names)

# expand
df_26_100.loc[:, 'cohort'] = 'medium'
df_26_100.loc[:, 'firm_size'] = np.random.randint(low=26,
                                                  high=100,
                                                  size=num_samples)

### Large firms

In [59]:
# the desired covariance matrix.
r = np.array([
        [  1.00, -0.20, -0.03,  0.11],
        [ -0.20,  1.00, -0.05, -0.09],
        [ -0.03, -0.05,  1.00,  0.05],
        [  0.11, -0.09,  0.05,  1.00]
    ])

# generate the random samples.
df_101_500 = pd.DataFrame(np.random.multivariate_normal(mu, r, size=num_samples),
                          columns=names)

# expand
df_101_500.loc[:, 'cohort'] = 'large'
df_501_.loc[:, 'firm_size'] = np.random.randint(low=101, high=500,
                                                size=num_samples)

### Very large firms

In [60]:
# the desired covariance matrix.
r = np.array([
        [  1.00, -0.15, -0.03,  0.11],
        [ -0.15,  1.00, -0.05, -0.09],
        [ -0.03, -0.05,  1.00,  0.05],
        [  0.11, -0.09,  0.05,  1.00]
    ])

# generate the random samples.
df_501_ = pd.DataFrame(np.random.multivariate_normal(mu, r, size=num_samples),
                       columns=names)

# expand
df_501_.loc[:, 'cohort'] = 'very large'
df_501_.loc[:, 'firm_size'] = np.random.randint(low=501, high=2000,
                                                size=num_samples)

#### Data preparation

In [61]:
df = pd.concat([df_1_5, df_6_25, df_26_100, df_101_500, df_501_],
               axis=0)
df.info()

df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   job_sat    5000 non-null   float64
 1   int_qui    5000 non-null   float64
 2   age        5000 non-null   float64
 3   org_tnr    5000 non-null   float64
 4   cohort     5000 non-null   object 
 5   firm_size  4000 non-null   float64
dtypes: float64(5), object(1)
memory usage: 273.4+ KB


,job_sat,int_qui,age,org_tnr,cohort,firm_size
0,-1.992759,-0.531744,0.208208,0.844195,micro,2.0
1,-2.746474,2.511886,-0.970228,0.053533,micro,4.0
2,-0.957536,-0.381626,0.631114,-0.382642,micro,4.0
3,0.725831,0.290553,1.543508,-0.499497,micro,1.0
4,-0.301695,0.276778,1.604164,-0.327978,micro,3.0


# Regression analysis

x ------> y
     ^
     |
     |
     z
     
Alternative 1:

+ 5 models, each of which applies to a certain cohort:
  - 5 betas [$\beta_{micro}$, $\beta_{small}$, ..... ]
  - no $\delta$ / no interaction effect
  
Alternative 2:
+ 1 model, whose equation is the one displayed at the top of this notebook
  - beta + delta

??

# Visualization

??